### Install~~~

In [1]:
pip install nltk


[notice] A new release of pip is available: 23.0.1 -> 25.0
[notice] To update, run: C:\Users\bubbl\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0
[notice] To update, run: C:\Users\bubbl\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
pip install python-docx

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0
[notice] To update, run: C:\Users\bubbl\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0
[notice] To update, run: C:\Users\bubbl\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0
[notice] To update, run: C:\Users\bubbl\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### 爬蟲

In [ ]:
import requests
import os
from bs4 import BeautifulSoup
from docx import Document

# 目標 Wikipedia 頁面
url = "https://en.wikipedia.org/wiki/Android_version_history"
headers = {"User-Agent": "Mozilla/5.0"}  # 避免被拒絕訪問
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# ✅ 使用者輸入要抓取的 Android 版本（例如 "Android 4.4" 或 "Android 1.0"）
target_version = input("請輸入要抓取的 Android 版本（如 Android 1.0, Android 4.4 等）：").strip()

# 找到所有 "wikitable" 表格
tables = soup.find_all("table", {"class": "wikitable"})

# 儲存結果
android_data = []

# 遍歷所有表格
for table in tables:
    heading = table.find_previous(["h2", "h3","h4"])  # 找到表格前面的標題
    version_title = heading.get_text(strip=True).replace("[edit]", "") if heading else "Unknown Version"

    # ✅ 檢查是否為目標版本
    if target_version.lower() in version_title.lower():
        print(f"🎯 找到目標版本：{version_title}")

        rows = table.find_all("tr")
        for row in rows[1:]:  # 跳過表頭
            cols = row.find_all("td")
            if len(cols) >= 3:  # 確保有 Version, Release Date, Features
                version = cols[0].get_text(strip=True)
                release_date = cols[1].get_text(strip=True)

                # 處理 Features 欄位
                feature_column = cols[2]
                features_list = [li.get_text(strip=True) for li in feature_column.find_all("li")]
                if not features_list:
                    features_list = [feature_column.get_text(strip=True)]  # 如果沒有 <li>，直接取文字

                android_data.append({
                    "Version": version if version else version_title,
                    "Release Date": release_date,
                    "Features": features_list
                })
        break  # ✅ 找到目標版本後就停止搜尋

# ✅ 確認是否找到資料
if not android_data:
    print(f"❌ 找不到版本：{target_version}")
else:
    # ✅ 將資料儲存成 DOCX 檔案
    doc = Document()
    doc.add_heading(f"Android Version: {target_version}", level=1)

    for entry in android_data:
        doc.add_heading(f"Version: {entry['Version']}", level=2)
        doc.add_paragraph(f"Release Date: {entry['Release Date']}")
        doc.add_paragraph("Features:")

        # 以列表形式輸出 Features
        for feature in entry["Features"]:
            doc.add_paragraph(f"- {feature}", style='List Bullet')

    # ✅ 設定要存檔的資料夾
    save_directory = r"C:\Users\bubbl\Desktop\爬蟲結果"

    # ✅ 確保資料夾存在（如果不存在就建立）
    os.makedirs(save_directory, exist_ok=True)

    # ✅ 產生完整的存檔路徑
    file_name = f"{target_version.replace(' ', '_')}_features_new.docx"
    output_path = os.path.join(save_directory, file_name)  # 👉 設定完整存檔路徑

    # ✅ 儲存 DOCX 檔案
    doc.save(output_path)

    print(f"✅ 資料已成功儲存至 {output_path}")

🎯 找到目標版本：Android 4.4 KitKat
✅ 資料已成功儲存至 C:\Users\bubbl\Desktop\爬蟲結果\4.4_features_new.docx


### TF-IDF值計算

#### Import

In [6]:
import re
import pandas as pd
import nltk
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from docx import Document

#### Downloads跟初始化字詞

In [204]:

# 下載 NLTK 必要資源（若尚未下載）
nltk.download('stopwords')
nltk.download('wordnet')

# 初始化詞形還原器與停用詞
lemmatizer = WordNetLemmatizer()
STOP_WORDS = set(stopwords.words('english'))
CUSTOM_STOP_WORDS_WHOLE = {"version", "release", "date", "android","however","etc","instead","wa","ability",
                            "upload","application","support","introduced","reintroduction","removed","blocked",
                            "designed","installation","older","new","access","user","fixed","improved",
                            "change","bug","fix","prevent","marshmallow","amendment","minor","native","improvement","miscellaneous"}


CUSTOM_STOP_WORDS_3      = {"version", "release", "date", "android","however","etc","instead","wa","ability",
                            "upload","application","support","introduced","reintroduction","removed","blocked",
                            "designed","installation","older","new","access","user","fixed","improved",
                            "system","screen"}

CUSTOM_STOP_WORDS_4_03 = {"bug","fix","improvement","improved","stability"}

CUSTOM_STOP_WORDS_2_3    = {"bug","fix","improvement","improved","support","native","user"}

CUSTOM_STOP_WORDS_7      = {"file","support","ability","application","overview","setting","added","access","redesigned","android"}


ALL_STOP_WORDS = list(STOP_WORDS) + list(CUSTOM_STOP_WORDS_4_03)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bubbl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bubbl\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### 讀取檔案

In [8]:
# ✅ Step 1: 讀取 DOCX 檔案，提取 Features 區塊
def extract_features_from_docx(file_path):
    doc = Document(file_path)
    features_list = []
    is_feature_section = False

    for para in doc.paragraphs:
        text = para.text.strip()

        # 開始提取 Features 區塊
        if text.lower() == "features:":
            is_feature_section = True
            continue

        # 結束 Features 區塊
        if is_feature_section and (text.lower().startswith("version") or text.lower().startswith("release date") or not text):
            break

        # 提取並清理 Features
        if is_feature_section and text:
            clean_text = text.replace("??", "").strip()
            features_list.append(clean_text)

    return features_list

#### 文本處理

In [9]:
# ✅ Step 2: 文本前處理（小寫化、標點移除、詞形還原）
def preprocess_text(text):

    text = re.sub(r'\[\w+\]', '', text)  # 移除方括号内的字母或数字
    
    # 处理特定的缩写，防止它们转换成小写后丢失字母
    text = re.sub(r'\bSMS\b', '[SMS]', text)  # 替换 SMS 为 [SMS]
    text = re.sub(r'\bMMS\b', '[MMS]', text)  # 替换 MMS 为 [MMS]
    
    text = re.sub(r'\bWi-Fi\b', 'wi_fi', text)  # 保留 wi-fi
    text = re.sub(r'\b802\.1x\b', '802_1X', text)  # 保留 802.1x

    # 在转换为小写并移除标点符号时，保留我们替换的缩写
    text = re.sub(r'[^\w\s\[\]]', ' ', text.lower())  # 转小写并移除标点符号，保留方括号

    text = re.sub(r'\s+', ' ', text).strip()  # 移除多余空白
    text = re.sub(r'\bapps?\b', 'application', text)
    text = re.sub(r'\bapplication\b', 'application', text)
    
    # 恢复之前替换的缩写
    text = re.sub(r'\[SMS\]', 'sms', text)
    text = re.sub(r'\[MMS\]', 'mms', text)

    words = text.split()

    # 詞形還原
    words = [lemmatizer.lemmatize(word, wordnet.NOUN) for word in words]

    return " ".join(words)

#### 正式計算

In [14]:
# ✅ Step 3: 計算 TF-IDF 分數
def calculate_tfidf(features_list):
    processed_features = [preprocess_text(feature) for feature in features_list]

    # 計算 TF-IDF
    vectorizer = TfidfVectorizer(stop_words=ALL_STOP_WORDS) 
    tfidf_matrix = vectorizer.fit_transform(processed_features)

    # 轉換為 DataFrame
    feature_names = vectorizer.get_feature_names_out()
    tfidf_scores = tfidf_matrix.toarray().mean(axis=0)  # 平均 TF-IDF 分數
    df_tfidf = pd.DataFrame({'Word': feature_names, 'TF-IDF Score': tfidf_scores})

    # 按 TF-IDF 分數排序
    df_tfidf_sorted = df_tfidf.sort_values(by='TF-IDF Score', ascending=False).reset_index(drop=True)
    return df_tfidf_sorted

#### 修改版本、結果與存檔

In [203]:
# ✅ 主程式：執行流程
file_path = 'C:/Users/bubbl/Desktop/爬蟲結果/待分析/2.3_features.docx'  # 更新為你的檔案路徑
features_list = extract_features_from_docx(file_path)
df_tfidf_sorted = calculate_tfidf(features_list)

# ✅ 顯示前 10 個最高 TF-IDF 分數的詞語
print("\n🔹 前 10 個最高 TF-IDF 值的詞語：")
print(df_tfidf_sorted.head(100))

# 轉為 DataFrame 並存回 CSV
output_path = "C:/Users/bubbl/Desktop/TF-IDF結果/Done_new/tfidf_scores-2.3.csv"  # 替換成你要存放的實際路徑
df_tfidf_sorted.to_csv(output_path, index=False)

print(f"CSV 文件已儲存至: {output_path}")


🔹 前 10 個最高 TF-IDF 值的詞語：
        Word  TF-IDF Score
0      nexus      0.105263
1      audio      0.048417
2     device      0.046311
3      input      0.042909
4   enhanced      0.041965
..       ...           ...
95  embedded      0.014184
96     field      0.014184
97      near      0.014184
98       tag      0.014184
99   sticker      0.014184

[100 rows x 2 columns]
CSV 文件已儲存至: C:/Users/bubbl/Desktop/TF-IDF結果/Done_new/tfidf_scores-2.3.csv


#### 爬蟲2 

In [80]:
import requests
import os
from bs4 import BeautifulSoup
from docx import Document

# 目標 Wikipedia 頁面
url = "https://en.wikipedia.org/wiki/Android_version_history"
headers = {"User-Agent": "Mozilla/5.0"}  # 避免被拒絕訪問
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# ✅ 使用者輸入大版本號，例如 "4.4"
base_version = input("請輸入要抓取的 Android 版本（如 1.0, 4.4 等）：").strip()

# 找到所有 "wikitable" 表格
tables = soup.find_all("table", {"class": "wikitable"})

# 儲存所有匹配的版本
matched_versions = {}

# 遍歷所有表格
for table in tables:
    heading = table.find_previous(["h2", "h3","h4"])  # 找到表格前面的標題
    version_title = heading.get_text(strip=True).replace("[edit]", "") if heading else "Unknown Version"

    rows = table.find_all("tr")
    for row in rows[1:]:  # 跳過表頭
        cols = row.find_all("td")
        if len(cols) >= 3:  # 確保有 Version, Release Date, Features
            version = cols[0].get_text(strip=True)

            # ✅ 只抓取以 base_version 開頭的版本（如 4.4、4.4.1、4.4.2）
            if version.startswith(base_version):
                matched_versions[version] = row  # 存入匹配的版本

# ✅ 如果沒有匹配的版本，顯示錯誤訊息
if not matched_versions:
    print(f"❌ 找不到版本：{base_version}")
    exit()

# ✅ 如果有多個匹配的版本，讓使用者選擇
if len(matched_versions) > 1:
    print("\n找到以下版本，請選擇：")
    version_list = list(matched_versions.keys())
    for i, v in enumerate(version_list):
        print(f"{i + 1}. {v}")

    while True:
        try:
            choice = int(input("請輸入對應的數字："))
            if 1 <= choice <= len(version_list):
                target_version = version_list[choice - 1]
                break
            else:
                print("❌ 輸入錯誤，請輸入正確的數字！")
        except ValueError:
            print("❌ 請輸入數字！")
else:
    # 只有一個匹配版本，直接選取
    target_version = list(matched_versions.keys())[0]

# ✅ 解析選定的版本資料
print(f"\n🎯 你選擇的版本是：{target_version}")
selected_row = matched_versions[target_version]
cols = selected_row.find_all("td")
release_date = cols[1].get_text(strip=True)

# 處理 Features 欄位
feature_column = cols[2]
features_list = [li.get_text(strip=True) for li in feature_column.find_all("li")]
if not features_list:
    features_list = [feature_column.get_text(strip=True)]  # 如果沒有 <li>，直接取文字

# ✅ 儲存到 DOCX
doc = Document()
doc.add_heading(f"Android Version: {target_version}", level=1)
doc.add_paragraph(f"Release Date: {release_date}")
doc.add_paragraph("Features:")

for feature in features_list:
    doc.add_paragraph(f"- {feature}", style='List Bullet')

# ✅ 設定存檔路徑
save_directory = r"C:\Users\bubbl\Desktop\爬蟲結果"
os.makedirs(save_directory, exist_ok=True)
file_name = f"{target_version.replace(' ', '_')}_features.docx" 
output_path = os.path.join(save_directory, file_name)

# ✅ 儲存 DOCX 檔案
doc.save(output_path)

print(f"✅ 資料已成功儲存至 {output_path}")



🎯 你選擇的版本是：3.2.6
✅ 資料已成功儲存至 C:\Users\bubbl\Desktop\爬蟲結果\3.2.6_features.docx
